In [1]:
# Import our dependencies
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf



**Read the data - information from Kaggle**

In [2]:
# Specify the file path
file_path = r'C:\Users\aumek\OneDrive\Desktop\Git_Personnal_Rep\Resource_mustang\merged_data.csv'

# Read the CSV file into pandas
df = pd.read_csv(file_path)
df.head(5)


C:\Users\aumek\AppData\Local\Temp\ipykernel_29992\1200018108.py:5: DtypeWarning: Columns (11,19,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Unnamed: 0,rid,course,time,date,title,rclass,band,ages,distance,...,OR,father,mother,gfather,runners,margin,weight,res_win,res_place,price
0,0,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Pour Moi,Laughing Water,Duke Of Marmalade,9.0,1.199095,66.0,1.0,1.0,NaN
1,1,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,110.0,Getaway,Shouette,Sadler's Wells,9.0,1.199095,65.0,0.0,1.0,NaN
2,2,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Vale Of York,Finnmark,Halling,9.0,1.199095,72.0,0.0,1.0,NaN
3,3,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Sixties Icon,Highland Jig,Norse Dancer,9.0,1.199095,72.0,0.0,0.0,NaN
4,4,10312,Fakenham,02:55,20/01/01,Happy New Year Maiden Hurdle (Div I),Class 4,NaN,4yo+,2m,...,NaN,Iffraaj,Hallowed Park,Barathea,9.0,1.199095,72.0,0.0,0.0,NaN


In [3]:
 # Get a list of all of our columns for easy reference
df.columns

Index(['Unnamed: 0', 'rid', 'course', 'time', 'date', 'title', 'rclass',
       'band', 'ages', 'distance', 'condition', 'hurdles', 'prizes',
       'winningTime', 'prize', 'metric', 'countryCode', 'ncond', 'class',
       'currency', 'horseName', 'age', 'saddle', 'decimalPrice', 'isFav',
       'trainerName', 'jockeyName', 'position', 'positionL', 'dist',
       'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'RPR',
       'TR', 'OR', 'father', 'mother', 'gfather', 'runners', 'margin',
       'weight', 'res_win', 'res_place', 'price'],
      dtype='object')

**Preparing the data For Model1 - where we are predicting if the 'favorite' horse wins, we used two datasets from "Kaggle" as listed below:**

*dataset 1 = races_* columns description: 
- rid - Race id; 
- Distance
- winningTime - Best time shown
- metric - Distance in meters
- class - class type (created from rclass feature).

*dataset 2 = horses_* columns description: 
- rid - Race id (column used to merge the 2 datasets)
- age - Horse age; 
- saddle - Saddle # where horse starts; 
- isFav - Was horse favorite before start? Can be more then one fav in a race;  
- position - Finishing position, 40 if horse didn't finish; 
- positionL - how far a horse has finished from the pursued horse, horses corpses; 
- outHandicap - Handicap; 
- RPR - RP Rating; 
- TR - Topspeed; 
- OR - Official Rating father - Horse's Father name; 
- runners - Runners total; 
- weight - Horse weight in kg; 
- res_win - Horse won or not; 


# **Data Cleanup**

In [4]:
# Drop columns
clean_df = df.loc[:, [ 'rid', 'winningTime', 'metric', 'ncond', 'class',
       'age', 'decimalPrice', 'isFav','position', 'positionL', 'distance','outHandicap', 'RPR','saddle',
       'TR', 'OR', 'runners', 'weight', 'res_win']]
clean_df.head(5)

,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4.0,0.076923,0.0,1.0,NaN,2m,NaN,103.0,9.0,103.0,NaN,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6.0,0.444444,1.0,2.0,15,2m,NaN,96.0,7.0,89.0,110.0,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7.0,0.019608,0.0,3.0,5.5,2m,NaN,97.0,5.0,88.0,NaN,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6.0,0.230769,0.0,4.0,10,2m,NaN,89.0,4.0,75.0,NaN,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7.0,0.038462,0.0,5.0,5.5,2m,NaN,82.0,6.0,67.0,NaN,9.0,72.0,0.0


In [5]:
import re

#remove special characters and normalize distance to show all as 'meters'

# Convert 'dist' column to string data type
clean_df['distance'] = clean_df['distance'].astype(str)

# Define the function to clean the distance values
def clean_distance(distance):
    if re.search(r'\d+m', distance):
        meters = re.findall(r'\d+', distance)
        return int(meters[0])
    elif re.search(r'\d+f', distance):
        furlongs = re.findall(r'\d+', distance)
        meters = int(furlongs[0]) * 201
        return meters
    elif re.search(r'\dm\d+f', distance):
        meters = re.findall(r'\d+m', distance)
        furlongs = re.findall(r'\d+f', distance)
        meters = int(meters[0].strip('m'))
        furlongs = int(furlongs[0].strip('f')) * 201
        return meters + furlongs
    else:
        return distance

# Apply the clean_distance function to the 'dist' column
clean_df['distance'] = clean_df['distance'].apply(clean_distance)

# Print the cleaned DataFrame
clean_df.head(5)


,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4.0,0.076923,0.0,1.0,NaN,2,NaN,103.0,9.0,103.0,NaN,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6.0,0.444444,1.0,2.0,15,2,NaN,96.0,7.0,89.0,110.0,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7.0,0.019608,0.0,3.0,5.5,2,NaN,97.0,5.0,88.0,NaN,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6.0,0.230769,0.0,4.0,10,2,NaN,89.0,4.0,75.0,NaN,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7.0,0.038462,0.0,5.0,5.5,2,NaN,82.0,6.0,67.0,NaN,9.0,72.0,0.0


In [6]:
#to verify the data type
clean_df.dtypes

rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age             float64
decimalPrice    float64
isFav           float64
position        float64
positionL        object
distance         object
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object

In [7]:
#changing datatype for 'positionL' and 'dist' to numeric
clean_df['positionL'] = pd.to_numeric(clean_df['positionL'], errors='coerce')
clean_df['distance'] = pd.to_numeric(clean_df['distance'], errors='coerce')

# Verify the updated data types
print(clean_df.dtypes)


rid               int64
winningTime     float64
metric          float64
ncond             int64
class             int64
age             float64
decimalPrice    float64
isFav           float64
position        float64
positionL       float64
distance        float64
outHandicap     float64
RPR             float64
saddle          float64
TR              float64
OR              float64
runners         float64
weight          float64
res_win         float64
dtype: object


In [8]:
import numpy as np

# Specify the columns you want to drop NA values
columns_to_drop_na = ['isFav', 'res_win']

# Specify the values representing blanks in those columns
blank_values = ['', ' ', '-']

# Replace specific blank values with NaN
clean_df.replace(blank_values, np.nan, inplace=True)

# Drop rows with NA values in the specified columns
clean_df.dropna(subset=columns_to_drop_na, inplace=True)

# Reset the index
clean_df.reset_index(drop=True, inplace=True)


In [9]:
# save to csv to visualize data
clean_df.to_csv('Resources/clean_data_model1_v2.csv', index=False)


In [10]:
# Count the remaining rows and columns in the DataFrame
num_rows, num_columns = clean_df.shape

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)



Number of rows: 149513
Number of columns: 19


# **Preprocessing Model 1 - Does the Favorite Horse win?**

 Step 1: Read the clean_data_model1.csv data from the Resources folder into a Pandas DataFrame.

In [11]:
clean_m1_df = pd.read_csv("Resources/clean_data_model1.csv")
clean_m1_df.head(5)

,rid,winningTime,metric,ncond,class,age,decimalPrice,isFav,position,positionL,distance,outHandicap,RPR,saddle,TR,OR,runners,weight,res_win
0,10312,253.88,3218.0,10,4,4,0.076923,0.0,1,4.57028,2.0,2.541359,103.0,9.0,103.0,74.677073,9.0,66.0,1.0
1,10312,253.88,3218.0,10,4,6,0.444444,1.0,2,15.00000,2.0,2.541359,96.0,7.0,89.0,110.000000,9.0,65.0,0.0
2,10312,253.88,3218.0,10,4,7,0.019608,0.0,3,5.50000,2.0,2.541359,97.0,5.0,88.0,74.677073,9.0,72.0,0.0
3,10312,253.88,3218.0,10,4,6,0.230769,0.0,4,10.00000,2.0,2.541359,89.0,4.0,75.0,74.677073,9.0,72.0,0.0
4,10312,253.88,3218.0,10,4,7,0.038462,0.0,5,5.50000,2.0,2.541359,82.0,6.0,67.0,74.677073,9.0,72.0,0.0


In [31]:
#drop any NaN from the dataset
coverted_df = clean_m1_df.dropna()


In [32]:
# Determine the number of unique values in each column.
unique_application = clean_m1_df.nunique()
print(unique_application)

rid             5617
winningTime     4375
metric            49
ncond             15
class              8
age               14
decimalPrice     274
isFav              2
position          25
positionL        106
distance           7
outHandicap       16
RPR              169
saddle            29
TR               157
OR               151
runners           26
weight            33
res_win            2
dtype: int64


In [49]:
# Convert categorical data to numeric with `pd.get_dummies`
coverted_df= pd.get_dummies(clean_m1_df)

In [50]:
# Look at isFav value counts for binning - where 0=not fav; 1=fav
application_type_counts = coverted_df['isFav'].value_counts()
application_type_counts

isFav
0.0    33733
1.0     4841
Name: count, dtype: int64

In [51]:
# save to csv to visualize data
coverted_df.to_csv('Resources/coverted_df_m1.csv', index=False)

### Further steps in the model-building process: feature selection, model selection, training, and evaluation ###

In [52]:
#feature selection - In this case our target is if the favorite horse wins
y = coverted_df["isFav"]
X = coverted_df.drop(columns="isFav")

In [53]:
#splitting the data in train/test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(28930, 18)

In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train and Evaluate the Model
### Since our Model 1 is trying to answer does a  "favorite horse" wins? Yes or No, we are using a classification model 

In [55]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
coverted_df = tf.keras.models.Sequential()

# First hidden layer
coverted_df.add(tf.keras.layers.Dense(units=500, activation="relu", input_dim=X_train.shape[1]))

# Second hidden layer
coverted_df.add(tf.keras.layers.Dense(units=150, activation="sigmoid"))

# Output layer
coverted_df.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
coverted_df.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 500)               9500      
                                                                 
 dense_22 (Dense)            (None, 150)               75150     
                                                                 
 dense_23 (Dense)            (None, 1)                 151       
                                                                 
Total params: 84,801
Trainable params: 84,801
Non-trainable params: 0
_________________________________________________________________


In [56]:
# Compile the model
coverted_df.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(28930, 18)
(9644, 18)


In [57]:
coverted_df = tf.keras.Sequential([
    tf.keras.layers.Dense(80, input_dim=X_train.shape[1], activation="sigmoid"),
    tf.keras.layers.Dense(30, activation="sigmoid"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

coverted_df.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

coverted_df.fit(X_train_scaled, y_train, epochs=10)

# Scale the test data
X_test_scaled = X_scaler.transform(X_test)




Epoch 1/10
905/905 [==============================] - 1s 781us/step - loss: 0.2178 - accuracy: 0.9130
Epoch 2/10
905/905 [==============================] - 1s 773us/step - loss: 0.1297 - accuracy: 0.9414
Epoch 3/10
905/905 [==============================] - 1s 764us/step - loss: 0.1271 - accuracy: 0.9428
Epoch 4/10
905/905 [==============================] - 1s 786us/step - loss: 0.1254 - accuracy: 0.9431
Epoch 5/10
905/905 [==============================] - 1s 784us/step - loss: 0.1237 - accuracy: 0.9424
Epoch 6/10
905/905 [==============================] - 1s 808us/step - loss: 0.1230 - accuracy: 0.9419
Epoch 7/10
905/905 [==============================] - 1s 821us/step - loss: 0.1219 - accuracy: 0.9435
Epoch 8/10
905/905 [==============================] - 1s 801us/step - loss: 0.1214 - accuracy: 0.9425
Epoch 9/10
905/905 [==============================] - 1s 824us/step - loss: 0.1203 - accuracy: 0.9439
Epoch 10/10
905/905 [==============================] - 1s 890us/step - loss: 0.119

In [58]:
# Evaluate the model on the scaled test data
loss, accuracy = coverted_df.evaluate(X_test_scaled, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

302/302 [==============================] - 0s 700us/step - loss: 0.1146 - accuracy: 0.9448
Loss: 0.11460459232330322
Accuracy: 0.9448361396789551


In [59]:
# Make predictions on the test data
y_pred = coverted_df.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)

302/302 [==============================] - 0s 596us/step


In [60]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred_binary)
print("Precision:", precision)


Precision: 0.8110091743119267


In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97      8434
         1.0       0.81      0.73      0.77      1210

    accuracy                           0.94      9644
   macro avg       0.89      0.85      0.87      9644
weighted avg       0.94      0.94      0.94      9644

